In [ ]:
## 16-08-2024
# I was alarmed because this output gives a lot more pairs than chymera has (688) but it turns out that those extra pairs are single targets#
# such as ABO-ABO. As instructed by Gokbag, this is the correct way to do this 
# NO other dataset has this issue

In [ ]:
%run Horlbeck_Functions.ipynb


In [ ]:

chymera = pd.read_csv('../InputData/Chymera/chymera.csv', keep_default_na=False)
chymera = chymera.reset_index()#


chymera_RPE1 = chymera.copy()
chymera_RPE1 = chymera_RPE1[['gene1', 'gene2', 'Cas9.Guide', 'Cpf1.Guide', 'RPE1.T18A', 'RPE1.T18B', 'RPE1.T18C', 'RPE1.T24A', 'RPE1.T24B', 'RPE1.T24C',
                              'RPE1.T0']]
chymera_RPE1

In [ ]:
#A second-generation hgRNA library was designed in which the ~5,000 highestexpressed
#genes across a panel of human cell lines (HAP1, RPE1, HEK293T,
#HCT116, HeLa and A375) were targeted either with Cas9 (paired with an
#intergenic-targeting Lb-Cas12a guide), Lb-Cas12a (paired with an intergenictargeting
#Cas9 guide) or simultaneously with both Cas9 and Lb-Cas12a guides
#(dual-targeting). Target sites for the dual-targeting constructs were spaced between
#107 base pairs (bp) and >946 kb (median distance, 6,863 bp). In addition, hgRNAs
#targeting intergenic and nontargeting sites were included as controls. This portion
#of the library included 61,888 hgRNA constructs.
#In the second part of the library, we targeted paralog gene pairs74 from gene
#families where both genes are expressed across a panel of human cell lines (HAP1,
#RPE1, HEK293T, HCT116, HeLa and A375). Of 1,381 strict human ohnolog
#families that have arisen from whole-genome duplications of vertebrate genomes74,
#we selected 1,344 paralogs (avoiding gene families with more than two paralogs).
#In addition we also targeted selected gene pairs of interest, some of which have
#previously been reported to interact genetically. All gene pairs were either targeted
#individually by Cas9 (paired with an intergenic-targeting Lb-Cas12a guide) or
#Lb-Cas12a (paired with an intergenic-targeting Cas9 guide), or with both Cas9
#and Lb-Cas12a guides paired in both possible orientations (dual-targeting). This
#portion of the library comprised 30,848 hgRNA constructs. The full contents of the
#human single-gene dual-targeting and paralog-targeting library can be found in
#Supplementary Table 5.


chymera_HAP1 = chymera.copy() # ONLY LTP 
chymera_HAP1  = chymera_HAP1[['gene1', 'gene2', 'Cas9.Guide', 'Cpf1.Guide', 'HAP1.T18A', 'HAP1.T18B', 'HAP1.T18C',
                              'HAP1.T0', 'Library', 'Cas9.Guide.Type', 'Cpf1.Guide.Type']]

chymera_HAP1.rename(columns={ 'Cas9.Guide': 'sgRNA_guide_name_g1',
                             'Cpf1.Guide': 'sgRNA_guide_name_g2',
                             'gene1': 'sgRNA_target_name_g1',
                             'gene2': 'sgRNA_target_name_g2'} , inplace = True)

chymera_HAP1[['sgRNA_guide_seq_g1', 'sgRNA_guide_seq_g2']] = chymera_HAP1[['sgRNA_guide_name_g1', 'sgRNA_guide_name_g2']]
chymera_HAP1
chymera_HAP1.loc[ chymera_HAP1['sgRNA_target_name_g1']== "NegControl" , 'sgRNA_target_name_g1' ] = 'CONTROL'
chymera_HAP1.loc[ chymera_HAP1['sgRNA_target_name_g2']== "NegControl" , 'sgRNA_target_name_g2' ] = 'CONTROL'
chymera_HAP1

chymera_HAP1['count_replicates'] = chymera_HAP1['HAP1.T0'].astype(str)  +  ";" + chymera_HAP1['HAP1.T18A'].astype(str) + ";" + chymera_HAP1['HAP1.T18B'].astype(str) + ";" + chymera_HAP1['HAP1.T18C'].astype(str)
chymera_HAP1['cell_line_origin'] = "HAP1"
chymera_HAP1['study_conditions'] = "T0;T18A;T18B;T18C"

chymera_HAP1['study_origin'] = 'Chymera'


chymera_HAP1.loc[chymera_HAP1['sgRNA_target_name_g2'] == 'None', 'sgRNA_target_name_g2'] = chymera_HAP1.loc[chymera_HAP1['sgRNA_target_name_g2'] == 'None', 'sgRNA_target_name_g1']






chymera_HAP1

chymera_HAP1.loc[chymera_HAP1['sgRNA_target_name_g2'] == 'None', 'sgRNA_target_name_g2'] = chymera_HAP1.loc[chymera_HAP1['sgRNA_target_name_g2'] == 'None', 'sgRNA_target_name_g1']
chymera_HAP1.loc[(chymera_HAP1['sgRNA_target_name_g1'] == "CONTROL") | (chymera_HAP1['sgRNA_target_name_g2'] == "CONTROL"), "target_type"] = "Control"


## THIS IS TOLD TO ME BY GOKBAG @ SLKB
##Target_type: Can be ‘Dual’ (two genes targeted), can be ‘Single’ (gene + control or gene + gene, where only one gene was targeted with both sgRNAs), or ‘Control’ (control + control)
## We also set two guides that target the same sgRNA as ‘Single’ target, such as ‘ABO & ABO’ target would also be classified as a single target, as would ‘ABO & Control’ and ‘Control & ABO’. I don’t know if this would generate the error, but I wanted to mention it nonetheless.
chymera_HAP1['target_type'] = "Dual"
chymera_HAP1.loc[(chymera_HAP1['sgRNA_target_name_g1'] == 'CONTROL') | (chymera_HAP1['sgRNA_target_name_g2'] == 'CONTROL'), 'target_type'] = 'Single'
chymera_HAP1.loc[(chymera_HAP1['sgRNA_target_name_g1'] == chymera_HAP1['sgRNA_target_name_g2']), 'target_type'] = 'Single'
chymera_HAP1.loc[(chymera_HAP1['sgRNA_target_name_g1'] == 'CONTROL') & (chymera_HAP1['sgRNA_target_name_g2'] == 'CONTROL'), 'target_type'] = 'Control'
chymera_HAP1.loc[(chymera_HAP1['sgRNA_target_name_g1'] == 'NT') & (chymera_HAP1['sgRNA_target_name_g2'] == 'NT'), 'target_type'] = 'Control'

chymera_HAP1['T0_counts'] = chymera_HAP1['HAP1.T0'].astype(str) 
chymera_HAP1['T0_replicate_names'] =  'T0'#'
chymera_HAP1['TEnd_counts'] = chymera_HAP1['HAP1.T18A'].astype(str) + ";" + chymera_HAP1['HAP1.T18B'].astype(str) + ";" + chymera_HAP1['HAP1.T18C'].astype(str)
chymera_HAP1['TEnd_replicate_names'] = 'T18A' + ";" "T18B" + ";" + "T18C"

chymera_HAP1['gene_pair_id'] = chymera_HAP1['sgRNA_target_name_g1'] + "|" + chymera_HAP1['sgRNA_target_name_g2']
chymera_HAP1['gene_pair'] = chymera_HAP1['gene_pair_id']

chymera_HAP1['sgRNA_pair'] = chymera_HAP1['sgRNA_guide_name_g1'] + "|" + chymera_HAP1['sgRNA_guide_name_g2'] 
chymera_HAP1['gene_pair_orientation'] = chymera_HAP1.apply(lambda row: 'A_B' if row['sgRNA_target_name_g1'] > row['sgRNA_target_name_g2'] else 'B_A', axis=1)
chymera_HAP1 = chymera_HAP1[['sgRNA_guide_name_g1','sgRNA_guide_name_g2','sgRNA_target_name_g1','sgRNA_target_name_g2','count_replicates','cell_line_origin','study_conditions','study_origin','target_type','T0_counts','T0_replicate_names','TEnd_counts','TEnd_replicate_names','gene_pair','gene_pair_orientation']]

chymera_HAP1


In [ ]:
chymera_HAP1[(chymera_HAP1['sgRNA_target_name_g1'] == chymera_HAP1['sgRNA_target_name_g2']) & (chymera_HAP1['sgRNA_target_name_g1'] != "NT") & (chymera_HAP1['sgRNA_target_name_g1'] != "CONTROL")].target_type.unique()

In [ ]:
# This was run on HPC
s1 = run_horlbeck_score(chymera_HAP1, 'chymera', "chymera_HAP1", do_preprocessing= True, re_run=True, filterThreshold= 0)


In [ ]:
chymera_RPE1 = chymera.copy() # ONLY LTP 
chymera_RPE1  = chymera_RPE1[['gene1', 'gene2', 'Cas9.Guide', 'Cpf1.Guide', 'RPE1.T24A', 'RPE1.T24B', 'RPE1.T24C',
                              'RPE1.T0', 'Library', 'Cas9.Guide.Type', 'Cpf1.Guide.Type']]

chymera_RPE1.rename(columns={ 'Cas9.Guide': 'sgRNA_guide_name_g1',
                             'Cpf1.Guide': 'sgRNA_guide_name_g2',
                             'gene1': 'sgRNA_target_name_g1',
                             'gene2': 'sgRNA_target_name_g2'} , inplace = True)

chymera_RPE1[['sgRNA_guide_seq_g1', 'sgRNA_guide_seq_g2']] = chymera_RPE1[['sgRNA_guide_name_g1', 'sgRNA_guide_name_g2']]
chymera_RPE1
chymera_RPE1.loc[ chymera_RPE1['sgRNA_target_name_g1']== "NegControl" , 'sgRNA_target_name_g1' ] = 'CONTROL'
chymera_RPE1.loc[ chymera_RPE1['sgRNA_target_name_g2']== "NegControl" , 'sgRNA_target_name_g2' ] = 'CONTROL'



chymera_RPE1['count_replicates'] = chymera_RPE1['RPE1.T0'].astype(str)  +  ";" + chymera_RPE1['RPE1.T24A'].astype(str) + ";" + chymera_RPE1['RPE1.T24B'].astype(str) + ";" + chymera_RPE1['RPE1.T24C'].astype(str)
chymera_RPE1['cell_line_origin'] = "RPE1"
chymera_RPE1['study_conditions'] = "T0;T24A;T24B;T24C"

chymera_RPE1['study_origin'] = 'Chymera'
chymera_RPE1[(chymera_RPE1['sgRNA_target_name_g1'] == "None") | (chymera_RPE1['sgRNA_target_name_g2'] == "None")] 

chymera_RPE1['target_type'] = np.where(
    (chymera_RPE1['sgRNA_target_name_g1'] == "None") | (chymera_RPE1['sgRNA_target_name_g2'] == "None"), 
    "Single", 
    np.NAN )

chymera_RPE1
chymera_RPE1.loc[chymera_RPE1['sgRNA_target_name_g2'] == 'None', 'sgRNA_target_name_g2'] = chymera_RPE1.loc[chymera_RPE1['sgRNA_target_name_g2'] == 'None', 'sgRNA_target_name_g1']


chymera_RPE1['target_type'] = "Dual"
chymera_RPE1.loc[(chymera_RPE1['sgRNA_target_name_g1'] == 'CONTROL') | (chymera_RPE1['sgRNA_target_name_g2'] == 'CONTROL'), 'target_type'] = 'Single'
chymera_RPE1.loc[(chymera_RPE1['sgRNA_target_name_g1'] == chymera_RPE1['sgRNA_target_name_g2']), 'target_type'] = 'Single'
chymera_RPE1.loc[(chymera_RPE1['sgRNA_target_name_g1'] == 'CONTROL') & (chymera_RPE1['sgRNA_target_name_g2'] == 'CONTROL'), 'target_type'] = 'Control'
chymera_RPE1.loc[(chymera_RPE1['sgRNA_target_name_g1'] == 'NT') & (chymera_RPE1['sgRNA_target_name_g2'] == 'NT'), 'target_type'] = 'Control'


chymera_RPE1['T0_counts'] = chymera_RPE1['RPE1.T0'].astype(str) 
chymera_RPE1['T0_replicate_names'] =  'T0'#'
chymera_RPE1['TEnd_counts'] = chymera_RPE1['RPE1.T24A'].astype(str) + ";" + chymera_RPE1['RPE1.T24B'].astype(str) + ";" + chymera_RPE1['RPE1.T24C'].astype(str)
chymera_RPE1['TEnd_replicate_names'] = 'T24A' + ";" "T24B" + ";" + "T24C"

chymera_RPE1['gene_pair_id'] = chymera_RPE1['sgRNA_target_name_g1'] + "|" + chymera_RPE1['sgRNA_target_name_g2']
chymera_RPE1['gene_pair'] = chymera_RPE1['gene_pair_id']

chymera_RPE1['sgRNA_pair'] = chymera_RPE1['sgRNA_guide_name_g1'] + "|" + chymera_RPE1['sgRNA_guide_name_g2'] 
chymera_RPE1['gene_pair_orientation'] = chymera_RPE1.apply(lambda row: 'A_B' if row['sgRNA_target_name_g1'] > row['sgRNA_target_name_g2'] else 'B_A', axis=1)
chymera_RPE1 = chymera_RPE1[['sgRNA_guide_name_g1','sgRNA_guide_name_g2','sgRNA_target_name_g1','sgRNA_target_name_g2','count_replicates','cell_line_origin','study_conditions','study_origin','target_type','T0_counts','T0_replicate_names','TEnd_counts','TEnd_replicate_names','gene_pair','gene_pair_orientation']]

chymera_RPE1


In [ ]:
s2 = run_horlbeck_score(chymera_RPE1, 'chymera', "chymera_RPE1", do_preprocessing= True, re_run=True, filterThreshold= 0)


In [ ]:
## CODE WAS RUN ON HPC
s = pd.concat([s1, s2], axis=0)
s.to_csv("Horlbeck Output/Horlbeck_Chymera.csv")